## Imports

In [3]:
import random
import numpy as np
random.seed(41)

c:\users\vansh\appdata\local\programs\python\python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\vansh\appdata\local\programs\python\python39\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\users\vansh\appdata\local\programs\python\python39\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Ingredients

In [4]:
n=3
G1 = [1 for i in range(n)]
G1 = np.asarray(G1).reshape(1, n)
print(G1)

# A (7, 4) hamming code
# Therefore G is (4, 7) matrix of form [I(4x4) P(4*3)]
G2 = [[1, 0, 0, 0, 1, 1, 0], 
      [0, 1, 0, 0, 0, 1, 1], 
      [0, 0, 1, 0, 1, 1, 1], 
      [0, 0, 0, 1, 1, 0, 1]]
G2 = np.asarray(G2)
print(G2)

# Corresponding H
H2 = [[1, 0, 1, 1, 1, 0, 0], 
     [1, 1, 1, 0, 0, 1, 0], 
     [0, 1, 1, 1, 0, 0, 1]]
H2 = np.asarray(H2)
print(H2)

[[1 1 1]]
[[1 0 0 0 1 1 0]
 [0 1 0 0 0 1 1]
 [0 0 1 0 1 1 1]
 [0 0 0 1 1 0 1]]
[[1 0 1 1 1 0 0]
 [1 1 1 0 0 1 0]
 [0 1 1 1 0 0 1]]


## Experiments

### Prelim

In [5]:
text = ""
with open('rickroll.txt') as f:
    tempLin = f.readlines()

for lin in tempLin:
    text += lin
# print(text)

binText = ''.join(format(ord(x), 'b').zfill(7) for x in text)
# print(binText)
M = len(binText)
print(M)

7756


### Common Functions

In [13]:
def incorporateErrTyp1(encodedStr, M, d):
    nonZ = random.sample(range(0, M), d)
    randBinErr = [0 for i in range(M)]
    for i in nonZ:
        randBinErr[i] = 1
    ErrStr = ""
    for k in randBinErr:
        ErrStr += str(k)
    assert(len(ErrStr) == len(encodedStr))
    y = int(ErrStr,2) ^ int(encodedStr,2)
    y = str(bin(y)[2:].zfill(len(ErrStr)))
    return y


def incorporateErrTyp2(encodedStr, M, d):
    startidx = random.randint(0, M-d+1)
    randBinErr = [0 for i in range(M)]
    for i in range(startidx, startidx+d):
        randBinErr[i] = 1
    # print(randBinErr)
    ErrStr = ""
    for kiter in randBinErr:
        ErrStr += str(kiter)
    assert(len(ErrStr) == len(encodedStr))
    y = int(ErrStr,2) ^ int(encodedStr,2)
    y = str(bin(y)[2:].zfill(len(ErrStr)))
    return y



def convToAsciiAndCompare(y):
    retrStr = ""
    cnt=0
    tots=0
    for i in range(0, len(y), 7):
        temp1 = y[i:i+7]
        retrStr += chr(int(temp1, 2))
        temp2 = binText[i:i+7]
        tots+=1
        if (temp1!=temp2):
            # print(temp1, temp2)
            cnt+=1
    return cnt, tots, retrStr


def encodeLC1(M, k):
    finEnc = []
    for i in range(0, M, k):
        tmpBlock = np.asarray(int(binText[i:i+k])).reshape(1, 1)
        bruh = np.matmul(tmpBlock, G1).reshape(-1).tolist()
        finEnc += bruh

    finEnc = ''.join([str(elem) for elem in finEnc])
    return finEnc


def encodeLC2(M, k):
    finEnc = []
    for i in range(0, M, k):
        tmpBlock = np.asarray([int(elem) for elem in binText[i:i+k]]).reshape(1,k)
        bruh = tmpBlock.dot(G2)%2
        bruh = bruh.reshape(-1).tolist()
        finEnc += bruh

    finEnc = ''.join([str(elem) for elem in finEnc]) #Length should be 7/4 times that of M
    return finEnc


def decodeLC1(y, n):
    y_ = ""
    detErr = 0
    for i in range(0, len(y), n):
        tmpblk = y[i:i+n]
        zerCnt = tmpblk.count('0')
        if zerCnt>n//2:
            y_+='0'
        else:
            y_+='1'
        if (zerCnt !=n):
            if (zerCnt!=0):
                detErr+=1
    return y_, detErr


def decodeLC2(y, n):
    H_ = np.transpose(H2) #(7x3)
    y_ = ""
    detErr = 0
    for i in range(0, len(y), n):
        tmpCod = y[i:i+n]
        tmpCodMat = []
        for kiter in tmpCod:
            tmpCodMat.append(int(kiter))
        synd = np.matmul(tmpCodMat, H_)%2
        if 1 not in synd:
            y_+=tmpCod[0:k]
        else:
            detErr+=1
            for idxt, elem in enumerate(H_):
                if (elem==synd).all():
                    idx = idxt
                    break
            E = [0 for i in range(n)]
            E[idx] = 1
            ErrFix = ""
            for kiter in E:
                ErrFix+=str(kiter)
            assert(len(ErrFix)==len(tmpCod))
            tmpy_ = int(ErrFix,2) ^ int(tmpCod,2)
            tmpy_ = str(bin(tmpy_)[2:].zfill(len(ErrFix)))
            y_ += tmpy_[0:k]
    return y_, detErr


def saveFile(fileName, Pystring):
    text_file = open(fileName, "w")
    text_file.write(Pystring)
    text_file.close()

### Experiment 1

In [7]:
D = [10, 100, 200, 500, 5000]
for d in D:
    
    #### ERROR ####
    y = incorporateErrTyp1(binText, M, d)
    
    #### FINDING AND PRINTING ERRORS ####
    cnt, tots, retrStr = convToAsciiAndCompare(y)
    print(cnt, tots, cnt*100/tots)
    
    #### SAVE FILE ####
    fileName = "rickroll_decode_Exp1_d"+str(d)+".txt"
    saveFile(fileName, retrStr)

10 1108 0.9025270758122743
97 1108 8.754512635379061
191 1108 17.23826714801444
404 1108 36.462093862815884
1107 1108 99.90974729241877


### Experiment 2

#### L1

In [9]:
n, k = 3, 1

#### ENCODING ####
finEnc = encodeLC1(M, k)
M_ = len(finEnc)

D = [10, 100, 200, 500, 5000]
for d in D:
    
    #### ERROR ####
    y = incorporateErrTyp1(finEnc, M_, d)
    
    #### DECODING ####
    y_, detErr = decodeLC1(y, n)    
    
    #### FINDING AND PRINTING ERRORS ####
    remErr = sum(1 for a, b in zip(y_, binText) if a != b)
    cnt, tots, retrStr = convToAsciiAndCompare(y_)
    print(detErr, detErr*100/len(y_), remErr, remErr*100/len(y_), cnt, tots, cnt*100/tots)
    
    #### SAVE FILE ####
    fileName = "rickroll_decode_Exp2_L1_d"+str(d)+".txt"
    saveFile(fileName, retrStr)

10 0.12893243940175347 0 0.0 0 1108 0.0
100 1.2893243940175347 0 0.0 0 1108 0.0
199 2.5657555440948943 1 0.012893243940175348 1 1108 0.09025270758122744
486 6.266116554925219 14 0.18050541516245489 14 1108 1.263537906137184
3895 50.21918514698298 943 12.158329035585353 649 1108 58.57400722021661


#### L2

In [14]:
n, k = 7, 4
# G = (4x7)
# H = (3x7)

#### ENCODING ####
finEnc = encodeLC2(M, k)
M_ = len(finEnc)

D = [10, 100, 200, 500, 5000]
for d in D:
    
    #### ERROR ####
    y = incorporateErrTyp1(finEnc, M_, d)
    
    #### DECODING ####
    y_, detErr = decodeLC2(y, n)
    
    #### FINDING AND PRINTING ERRORS ####
    remErr= sum(1 for a, b in zip(y_, binText) if a != b)
    cnt, tots, retrStr = convToAsciiAndCompare(y_)
    print(detErr, detErr*100/len(y_), remErr, remErr*100/len(y_), cnt, tots, cnt*100/tots)
    
    #### SAVE FILE ####
    fileName = "rickroll_decode_Exp2_L2_d"+str(d)+".txt"
    saveFile(fileName, retrStr)

10 0.12893243940175347 0 0.0 0 1108 0.0
97 1.2506446621970089 5 0.06446621970087674 3 1108 0.27075812274368233
187 2.41103661681279 18 0.23207839092315627 13 1108 1.1732851985559567
449 5.789066529138731 78 1.0056730273336771 52 1108 4.693140794223827
1666 21.48014440433213 3064 39.504899432697265 1057 1108 95.3971119133574


### Experiment 3

#### L1

In [15]:
n, k = 3, 1

#### ENCODING ####
finEnc = encodeLC1(M, k)
M_ = len(finEnc)

D = [10, 100, 200, 500]
for d in D:
    
    #### ERROR ####
    y = incorporateErrTyp2(finEnc, M_, d)
    
    #### DECODING ####
    y_, detErr = decodeLC1(y, n)
    
    #### FINDING AND PRINTING ERRORS ####
    remErr = sum(1 for a, b in zip(y_, binText) if a != b)
    cnt, tots, retrStr = convToAsciiAndCompare(y_)
    print(detErr, detErr*100/len(y_), remErr, remErr*100/len(y_), cnt, tots, cnt*100/tots)
    
    #### SAVE FILE ####
    fileName = "rickroll_decode_Exp3_L1_d"+str(d)+".txt"
    saveFile(fileName, retrStr)

2 0.025786487880350695 4 0.05157297576070139 1 1108 0.09025270758122744
2 0.025786487880350695 34 0.43837029396596183 6 1108 0.5415162454873647
1 0.012893243940175348 67 0.8638473439917483 10 1108 0.9025270758122743
1 0.012893243940175348 167 2.153171738009283 25 1108 2.256317689530686
1 0.012893243940175348 1667 21.493037648272306 239 1108 21.57039711191336


#### L2

In [16]:
n, k = 7, 4
# G = (4x7)
# H = (3x7)

#### ENCODING ####
finEnc = encodeLC2(M, k)
M_ = len(finEnc)

D = [10, 100, 200, 500]
for d in D:
    #### ERROR ####
    y = incorporateErrTyp2(finEnc, M_, d)
    
    #### DECODING ####
    y_, detErr = decodeLC2(y, n)
    
    #### FINDING AND PRINTING ERRORS ####
    remErr = sum(1 for a, b in zip(y_, binText) if a != b)
    cnt, tots, retrStr = convToAsciiAndCompare(y_)
    print(detErr, detErr*100/len(y_), remErr, remErr*100/len(y_), cnt, tots, cnt*100/tots)
    
    #### SAVE FILE ####
    fileName = "rickroll_decode_Exp3_L2_d"+str(d)+".txt"
    saveFile(fileName, retrStr)

2 0.025786487880350695 5 0.06446621970087674 2 1108 0.18050541516245489
1 0.012893243940175348 59 0.7607013924703455 9 1108 0.8122743682310469
2 0.025786487880350695 116 1.4956162970603404 18 1108 1.6245487364620939
2 0.025786487880350695 284 3.661681279009799 42 1108 3.7906137184115525


### Experiment 4

#### L1

In [23]:
n, k = 3, 1

#### ENCODING ####
finEnc = encodeLC1(M, k)
M_ = len(finEnc)
rot_fac = M_//2 # Because why not

#### PERMUTE: RIGHT SHIFT BY ROT_FAC ####
# finEncRot = finEnc[len(finEnc)-rot_fac : ] + finEnc[0 : len(finEnc)-rot_fac]
finEncRot = ""
permut = random.sample(range(0, M_), M_)
for el in permut:
    finEncRot+=finEnc[el]
    
D = [10, 100, 200, 500]
for d in D:
    
    #### ERROR ####
    y = incorporateErrTyp2(finEncRot, M_, d)
    
    ### INVERSE PERMUTE: RIGHT SHIFT BY ROT_FAC ####
    # y_ = y[rot_fac : ] + y[0 : rot_fac]
    y_tmp = ["," for _ in range(len(y))]
    for a,b in zip(y, permut):
        y_tmp[b] = a
    y_ = ""
    for el in y_tmp:
        y_+=el
    
    #### DECODING ####
    y_, detErr = decodeLC1(y_, n)
    
    #### FINDING AND PRINTING ERRORS ####
    remErr = sum(1 for a, b in zip(y_, binText) if a != b)
    cnt, tots, retrStr = convToAsciiAndCompare(y_)
    print(detErr, detErr*100/len(y_), remErr, remErr*100/len(y_), cnt, tots, cnt*100/tots)
    
    #### SAVE FILE ####
    fileName = "rickroll_decode_Exp4_L1_d"+str(d)+".txt"
    saveFile(fileName, retrStr)

10 0.12893243940175347 0 0.0 0 1108 0.0
99 1.2764311500773595 1 0.012893243940175348 1 1108 0.09025270758122744
199 2.5657555440948943 1 0.012893243940175348 1 1108 0.09025270758122744
485 6.253223310985044 15 0.1933986591026302 15 1108 1.3537906137184115


#### L2

In [26]:
n, k = 7, 4
# G = (4x7)
# H = (3x7)

#### ENCODING ####
finEnc = encodeLC2(M, k)
M_ = len(finEnc)
rot_fac = M_//2 # Because why not

#### PERMUTE: RIGHT SHIFT BY ROT_FAC ####
# finEncRot = finEnc[len(finEnc)-rot_fac : ] + finEnc[0 : len(finEnc)-rot_fac]
finEncRot = ""
permut = random.sample(range(0, M_), M_)
for el in permut:
    finEncRot+=finEnc[el]

D = [10, 100, 200, 500]
for d in D:
    #### ERROR ####
    y = incorporateErrTyp2(finEncRot, M_, d)
    
    ### INVERSE PERMUTE: LEFT SHIFT BY ROT_FAC ####
    # y_ = y[rot_fac : ] + y[0 : rot_fac]
    y_tmp = ["," for _ in range(len(y))]
    for a,b in zip(y, permut):
        y_tmp[b] = a
    y_ = ""
    for el in y_tmp:
        y_+=el
    
    #### DECODING ####
    y_, detErr = decodeLC2(y_, n)
    
    #### FINDING AND PRINTING ERRORS ####
    remErr = sum(1 for a, b in zip(y_, binText) if a != b)
    cnt, tots, retrStr = convToAsciiAndCompare(y_)
    print(detErr, detErr*100/len(y_), remErr, remErr*100/len(y_), cnt, tots, cnt*100/tots)
    
    #### SAVE FILE ####
    fileName = "rickroll_decode_Exp4_L2_d"+str(d)+".txt"
    saveFile(fileName, retrStr)

10 0.12893243940175347 0 0.0 0 1108 0.0
96 1.2377514182568334 8 0.10314595152140278 5 1108 0.45126353790613716
193 2.488396080453842 12 0.15471892728210418 6 1108 0.5415162454873647
441 5.6859205776173285 92 1.1861784424961321 62 1108 5.595667870036101


## Self-Help

In [ ]:
"""
Linear code 2
7*message code
XOR
chunk of 7 -> 2
if codeword, multiply with H. C*H' = 0
Not 0 if error. 
(C + E)*H' = E*H' = syndrome
0100000
y = c+e
c = y+e
c = len(7)
first 4 bits

detect: 
L2: syndrome non-zero => error (Detection/correction)
For correction: comparison from actual 
"""